### Load the required modules

In [1]:
import vaex
import pandas as pd
import panel as pn
import math
import param
import os
import glob
import datetime

In [2]:
pn.extension()

### Vaex Classes

**`Deal with creation of required variables to load, generate DataFrame and render paginated Data`**

In [3]:
class Board(param.Parameterized):
    #Inital Class that houses the basic Dashboard params
    
     
    file_path = param.String(default="", doc="Enter File Path")                                     # param that takes file path
    load_file = param.Action(lambda x: x.param.trigger('load_file'))                                # param that triggers gen_df function
    page = param.Integer(1, bounds=(1, 30))                                                         # param that takes page number
    select_column = param.Selector(default = "--Select Column--", objects=['--Select Column--'])    # param that takes column to be selected for filtering

    
    def __init__(self, **params):
        super(Board, self).__init__(**params)
        self.df = pd.DataFrame()               # initializes empty pandas dataframe 
        self.total_length = 0                  # total length of dataframe
        self.per_page = 30                     # records per page
        self.last_page = 30                    # last page  
        self.data = self.df                    # copy of dataframe subsequently used to store the filtered dataframe
        self.data_page = self.df               # initializes the variable subsequently used for current page of dataframe
        
        
    #Generate DataFrame    
    @param.depends('load_file')
    def gen_df(self):
        if self.file_path != "":
            self.df = vaex.open(str(self.file_path))                                     # loads the file as a vaex dataframe
            self.total_length = len(self.df)                                             # modifies total length variable
            self.last_page = math.ceil(len(self.df) / 30)                                # modifies last page variable based on file size
            self.param.page.bounds = (1, self.last_page)                                 # modifies upper bound of page param based on size of dataframe
            self.param.select_column.default = self.df.get_column_names()[0]             # modifies the default value of the select column param
            self.param.select_column.objects = list(self.df.get_column_names())          # modifies the objects listed as part of the select column param 
            self.data = self.df                                                          # replaces data variable with the newly loaded data 
            self.data_page = self.df.take(range(0, self.per_page)).to_pandas_df()        # slices the loaded dataframe and passes the scliced dataframe to data page variable

    
    # following function is triggered on page change
    @param.depends('page')
    def paginate(self):
        if self.file_path != "":
            self.from_item = ((self.page) - 1) * 30                                             # calculates starting index for current page     
            self.to_item = (self.page * 30)                                                     # calculates ending index for current page 
            self.data_page = self.data.take(range(self.from_item, self.to_item)).to_pandas_df() # slices the datframe to get records for current page
            df_widget = pn.widgets.DataFrame(self.data_page, name= 'DataSlice', width=1200)     # feeds the sliced dataframe to dataframe widget
            return df_widget
    
    @param.depends('load_file')
    def view(self):
        return pn.Row(self.gen_df, self.paginate)    #renders generated df and calls the paginate function
        

**`Deal with generation of Filters and Queries used to Slice the DataFrame`**

In [4]:
class Filter(Board):
     # Class Deals with Filtering the dataframe
        
    query = param.String(default="", doc="Query")                        # param that takes filter to be applied to the column selected
    add_filter = param.Action(lambda x: x.param.trigger('add_filter'))   # param that triggers add_query function

    _filters = {}                                                        # dict that stores all the queries

    
    # Function that adds the name of column as key to _filters dict every time a new column in selected
    @param.depends('select_column')                                      
    def new_query(self):
        self.query = ""
        if self.select_column not in self._filters.keys():
            self._filters[self.select_column] = []
    
    # adds the query string used to filter the selected column to _filters dict 
    @param.depends('add_filter')
    def add_query(self):
        if self.select_column not in self._filters.keys():
            self._filters[self.select_column] = []                          # initializes empty list to house the queries for the column selected 
        if self.select_column in self._filters.keys():                       
            if self.query not in self._filters[self.select_column]:         # if the enetered query of filter is not in list of queries for the selected column    
                self._filters[self.select_column].append(self.query)        # appends it to the list of queries
        if self.select_column in self._filters.keys() and self.query=="":   # if you select a column with the keu already in the filters dict and add a null string 
            self._filters[self.select_column] = []                          # clears the list of queries for the slected column

Helper Function to extract individual filter objects from the dict of filters

In [5]:
def extract_val(dictionary):
    ''' 
    
    This function extracts the individual values that can be applied as filters to a particular column from the filters dict. 
    Essentially creates a list where each item is a value that can be used to filter a column using slicing.
    
    NOT yet configured for use with ge and lt operators.
    
    '''
    
    vaex_filter_list = []
    for key, values in dictionary.items():
        if values != []:
            query_str = ""
            for value in values:
                if ',' in value:
                    for item in value.split(', '):
                        query_str += '(' + key +'=='+ item +')|'
                    query_str = query_str[:-1]
                else:
                    query_str += '(' + key +'=='+ value +')'
            vaex_filter_list.append(query_str)
    return vaex_filter_list

**`Deal with slicing the dataframe using the queries and downloading the sliced dataframe`**

In [6]:
class Slice(Filter):
    
    query_string = param.String(default="", doc="Query String")              # param that displays the consolidated string of queries applied to the dataframe
    apply_filter = param.Action(lambda x: x.param.trigger('apply_filter'))   # triggers the create_query_str function that also filters the dataframe
    download_file = param.Action(lambda x: x.param.trigger('download_file')) # download file button
    
    
    @param.depends('apply_filter')
    def create_query_st(self):
        qs = "".join(extract_val(self._filters))         # creates consolidates query string
        self.query_string = qs                           
        if qs != "":
            for i in extract_val(self._filters):         # iteratively applies each item from list of filters to slice the dataframe 
                self.data = self.data[self.data[i]]      
        else:
            self.data = self.df
            
    @param.depends('download_file')
    def export_data(self):
        self.data.export(f'data_.hdf5')

### Initiate Our Class render the Panel objects

In [7]:
Slicer = Slice()

In [8]:
svatdash = pn.Row(pn.Column(Slicer.param.file_path,
                            Slicer.param.load_file,
                            Slicer.param.download_file
                            ), 
                  Slicer.view,
                  pn.Column(Slicer.param.select_column,
                            Slicer.param.query,
                            Slicer.param.add_filter,
                            Slicer.param.query_string,
                            Slicer.param.apply_filter,
                            Slicer.param.page,
                            Slicer.new_query, Slicer.add_query, Slicer.create_query_st
                            )
                  )

In [20]:
for i in extract_val(Slicer._filters):
    print(i)

(State=='Maharashtra')|(State=='Bihar')
(Tmax==21)


In [10]:
svatdash

Row
    [0] Column
        [0] TextInput(name='File path', value='../data/rice_q1.csv')
        [1] Button(clicks=3, name='Load file')
        [2] Button(name='Download file')
    [1] ParamMethod(method)
    [2] Column
        [0] Select(options=OrderedDict([('Unnamed: 0'...]), value='Unnamed: 0')
        [1] TextInput(name='Query')
        [2] Button(name='Add filter')
        [3] TextInput(name='Query string')
        [4] Button(name='Apply filter')
        [5] IntSlider(end=7, name='Page', start=1, value=1, value_throttled=1)
        [6] ParamMethod(method)
        [7] ParamMethod(method)
        [8] ParamMethod(method)

### Serve the App

In [16]:
pn.serve(svatdash, websocket_origin='localhost', port=80)

INFO:MainThread:bokeh.server.server:Starting Bokeh server version 2.1.0 (running on Tornado 6.0.4)
INFO:MainThread:bokeh.server.tornado:User authentication hooks NOT provided (default user enabled)


Launching server at http://localhost:80


INFO:MainThread:tornado.access:200 GET / (::1) 335.00ms
INFO:MainThread:tornado.access:200 GET /static/js/bokeh.min.js?v=9f95464a3b695626394f06a80b4f1a60 (::1) 9.00ms
INFO:MainThread:tornado.access:200 GET /static/js/bokeh-tables.min.js?v=4b253f2902a0a75d9316cf4c37a3b15a (::1) 8.95ms
INFO:MainThread:tornado.access:200 GET /static/js/bokeh-gl.min.js?v=09cf0db4b301c631e2b8aae0b7bc5180 (::1) 29.00ms
INFO:MainThread:tornado.access:200 GET /static/extensions/panel/panel.min.js (::1) 39.01ms
INFO:MainThread:tornado.access:200 GET /static/js/bokeh-widgets.min.js?v=7c757501c076efba17bdce1dcb71f383 (::1) 50.00ms
INFO:MainThread:tornado.access:101 GET /ws (::1) 3.00ms
INFO:MainThread:bokeh.server.views.ws:WebSocket connection opened
INFO:MainThread:bokeh.server.views.ws:ServerConnection created
